In [1]:
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.models import Model
from numpy import genfromtxt 
from matplotlib import pyplot as plt 
%matplotlib inline 

train = genfromtxt('E:\\INSOFE\\AI and Deep Learning\\day1\\ANN\\train_solved.csv', delimiter=',') 
test = genfromtxt('E:\\INSOFE\\AI and Deep Learning\\day1\\ANN\\test_solved.csv', delimiter=',')

Using TensorFlow backend.


In [4]:
train = train/255.0
test = test/255.0

In [24]:
# Seperating the input data and labels
x_train = train[:,0:784] 
y_train = train[:,784]*255.0
x_test = test[:,0:784]
y_test = test[:,784]*255.0


In [25]:
from keras.utils.np_utils import to_categorical 
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)

In [8]:
from keras.models import Sequential
from keras.layers import Dense

In [9]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(784,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(784, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)

nb_epoch = 50
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['mse'])

In [10]:
history = autoencoder.fit(x_train, x_train,
                    epochs=5,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test,x_test),
                    verbose=1)


autoencoder.save('E:\\INSOFE\\AI and Deep Learning\\20171022_Batch31_CSE7321c_Lab02_DL\\DAE_l1_model.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 110s - loss: 0.0249 - mean_squared_error: 0.0249 - val_loss: 0.0124 - val_mean_squared_error: 0.0124
Epoch 2/5
60000/60000 [==============================] - 110s - loss: 0.0186 - mean_squared_error: 0.0186 - val_loss: 0.0112 - val_mean_squared_error: 0.0112
Epoch 3/5
60000/60000 [==============================] - 111s - loss: 0.0161 - mean_squared_error: 0.0161 - val_loss: 0.0102 - val_mean_squared_error: 0.0102
Epoch 4/5
60000/60000 [==============================] - 111s - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.0094 - val_mean_squared_error: 0.0094
Epoch 5/5
60000/60000 [==============================] - 112s - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.0090 - val_mean_squared_error: 0.0090


ImportError: cannot import name 'load_model'

In [11]:
from keras.models import load_model
autoencoder1 = load_model('E:\\INSOFE\\AI and Deep Learning\\20171022_Batch31_CSE7321c_Lab02_DL\\DAE_l1_model.h5')

In [14]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(x_train)

In [15]:
input_img1 = Input(shape=(1000,))
crrpt_img1 = Dropout(0.5)(input_img1)
encoded1 = Dense(1000, activation='sigmoid')(crrpt_img1)
decoded1 = Dense(1000, activation='sigmoid')(encoded1)

autoencoder1 = Model(input_img1,decoded1)

autoencoder1.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['binary_crossentropy'])

In [16]:
history = autoencoder1.fit(htrain_data, htrain_data,
                    epochs=2,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=1)


autoencoder1.save('E:\\INSOFE\\AI and Deep Learning\\20171022_Batch31_CSE7321c_Lab02_DL\\DAE_l2_model.h5')
from keras.models import load_model
autoencoder1 = load_model('E:\\INSOFE\\AI and Deep Learning\\20171022_Batch31_CSE7321c_Lab02_DL\\DAE_l2_model.h5')

Epoch 1/2
60000/60000 [==============================] - 136s - loss: 0.2104 - binary_crossentropy: 0.2104   
Epoch 2/2
60000/60000 [==============================] - 135s - loss: 0.2008 - binary_crossentropy: 0.2008   


In [27]:
# training a simple one hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(784,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(x_train ,y_train,
                    batch_size = 32,
                    epochs=1,
                    verbose=1)

Epoch 1/1
60000/60000 [==============================] - 127s - loss: 0.4647 - acc: 0.8508   


In [28]:
# Training with unsupervised initialization for layer-1 
# of an MLP using autoencoder weights

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(784,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

mlp.layers[1].set_weights(autoencoder.layers[2].get_weights())
mlp.layers[3].set_weights(autoencoder1.layers[2].get_weights())


history = mlp.fit(x_train ,y_train,
                    batch_size=batch_size,
                    epochs=3,
                    verbose=1,)

Epoch 1/3
60000/60000 [==============================] - 126s - loss: 0.2831 - acc: 0.9124   - ETA: 1s - loss:
Epoch 2/3
60000/60000 [==============================] - 126s - loss: 0.1450 - acc: 0.9552   
Epoch 3/3
60000/60000 [==============================] - 128s - loss: 0.1191 - acc: 0.9623   


In [29]:
mlp.evaluate(x_test,y_test)

 9888/10000 [============================>.] - ETA: 0s

[0.065083145525492728, 0.98050000000000004]